# Downloading Enron and Nazario Datasets


In [ ]:
import requests
import os, sys
import tarfile

sys.path.append(os.path.abspath(os.path.join("../..")))

In this section, we'll acquire the necessary datasets for this project. Note that the Enron dataset is substantial in size, so the download process might take a while, depending on your internet connection speed.


### Nazario (phishing)


In [ ]:
# URLs for the range of files
base_url = "https://monkey.org/~jose/phishing/phishing-"
start_year = 2015
end_year = 2022

# Directory to save the downloaded files
save_directory = "../../data/raw/phishing/"

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Loop through the range of years and download the files if they don't exist
for year in range(start_year, end_year + 1):
    file_path = os.path.join(save_directory, f"phishing-{year}.txt")

    # Check if the file already exists
    if not os.path.exists(file_path):
        url = f"{base_url}{year}"
        response = requests.get(url)

        if response.status_code == 200:
            # Save the file
            with open(file_path, "wb") as file:
                file.write(response.content)
            print(f"Downloaded: phishing-{year}")
        else:
            print(f"Failed to download: phishing-{year}")
    else:
        print(f"File already exists: phishing-{year}")
print("Download process completed.")

### Enron (legit)


In [ ]:
# URL for the file
url = "https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz"

# Directory to save the downloaded file
save_directory = "../../data/raw/"

# File name to save
file_name = os.path.join(save_directory, "enron_mail_20150507.tar.gz")


# Function to check if the file already exists
def file_exists(file_name):
    return os.path.exists(file_name)


# Function to track download progress
def download_file(url, file_name):
    if file_exists(file_name):
        print("File already exists. Skipping download.")
        return

    response = requests.get(url, stream=True)
    total_size = int(response.headers.get("content-length", 0))
    block_size = 1024  # 1 KB
    downloaded = 0

    with open(file_name, "wb") as file:
        for data in response.iter_content(block_size):
            file.write(data)
            downloaded += len(data)
            percent_complete = (downloaded / total_size) * 100
            print(
                f"\rDownload progress: {downloaded}/{total_size} bytes ({percent_complete:.2f}%)",
                end="",
            )

    print("\nDownload complete!")


# Download the file only if it doesn't exist
if not file_exists(file_name):
    download_file(url, file_name)
else:
    print("File already exists. Skipping download.")

Extracting the downloaded .tar.gz file and renaming the directory as "legit"


In [ ]:
# Directory paths
downloaded_file = "../../data/raw/enron_mail_20150507.tar.gz"
extracted_directory = "../../data/raw/"


# Function to check if the directory already exists
def directory_exists(directory):
    return os.path.exists(directory)


# Function to extract the contents of the downloaded file with progress tracking
def extract_with_progress(tar_file, extraction_dir):
    total_files = 0
    extracted_files = 0

    with tarfile.open(tar_file, "r:gz") as tar:
        total_files = len(tar.getmembers())  # Count all members in the archive

        for member in tar.getmembers():
            tar.extract(member, path=extraction_dir)
            extracted_files += 1
            progress = (extracted_files / total_files) * 100
            print(f"\rExtraction progress: {progress:.2f}%", end="")


# Extract only if the directory doesn't exist
if not directory_exists(extracted_directory + "legit/") and not directory_exists(
    extracted_directory + "maildir/"
):
    extract_with_progress(downloaded_file, extracted_directory)
    # Rename maildir/ to legit/ after extraction
    os.chmod(os.path.join(extracted_directory, "maildir"), 0o755)

    os.rename(
        os.path.join(extracted_directory, "maildir"),
        os.path.join(extracted_directory, "legit"),
    )
    print("\nExtraction complete!")
else:
    print("Extraction directory already exists. Skipping extraction.")